- Reference: https://tableau.github.io/server-client-python/docs/api-ref
- "HDC현대산업개발": "2a21240d-3f53-4f02-944d-b01647c3dd51"

In [1]:
import tableauserverclient as TSC
import pandas as pd
import numpy as np
import pymssql
from dateutil.relativedelta import relativedelta

f = open("D:/Github/Work/Tableau/datamart_password.txt", "r")
pw1 = f.readline()
f = open("D:/Github/Work/Tableau/tableau_server_password.txt", "r")
pw2 = f.readline()

pd.set_option("display.max_columns", None)

# SIgn In
tableau_auth = TSC.TableauAuth("6363", pw2)
server = TSC.Server("https://datalab.hdc-dvp.com", use_server_version=True)

In [2]:
conn = pymssql.connect(server="133.186.215.165", database="HDCMART", user="hdcmart_user", password=pw1, charset="UTF8")
query = f"""
SELECT id_sabun AS num, ds_hname AS name, ds_email AS email, ds_bonbu AS hq, ds_dept AS dept, ds_occupation AS job, DS_EMPTYPE AS sub
FROM DATAMART_DAAV_BASEINFO_DETAIL_ALL
where cd_corp = 'A101' and ds_retire = '재직';
"""
hr = pd.read_sql(query, conn)
hr.columns = hr.columns.str.lower()
for col in hr.columns:
    if pd.api.types.is_string_dtype(hr[col]):
        hr[col] = hr[col].str.encode("latin-1", errors="ignore").str.decode("euc-kr", errors="ignore")
        
hr.loc[hr["num"]=="1", "num"] = "0001"

empls = {str(row["num"]):(row["hq"], row["dept"], row["job"], row["name"]) for _, row in hr.iterrows()}

req_opts = TSC.RequestOptions(pagesize=1000)

# Create or Remove Users

In [3]:
with server.auth.sign_in(tableau_auth):
    all_users, _ = server.users.get(req_options=req_opts)

users_to_create = hr[(hr["job"]=="회장") | (hr["job"]=="부회장") | (hr["job"]=="대표이사") | (hr["job"]=="본부장") | ((hr["sub"]!="정보기술/CS외주") & (hr["job"]=="팀장")) | (hr["job"]=="실장") | (hr["job"]=="그룹장") | (hr["job"]=="부문장") | (hr["job"]=="단장") | (hr["job"]=="PL") | (hr["job"]=="PM") | (hr["job"]=="TFT장") | (hr["job"]=="건축소장") | (hr["job"]=="토목소장") | ((hr["dept"]=="경영분석팀") & (hr["sub"]=="일반직") & (hr["job"]=="팀원") & (~hr["num"].isin(["4113"]))) | ((hr["dept"]=="HR혁신팀") & (hr["sub"]=="일반직") & (hr["job"]=="팀원")) | (hr["dept"]=="상근고문") | (hr["num"].isin([
"6363", "5899", "5947", # 관리자
"4100435", "4001380", "4100222", # 이재호, 김아현, 김혜영
"4001341", "4001360", "4001386", # 비서
"1101291", "6294", "6111", "6048", "6403", "6032", # 디지털플랫폼팀 or BI 관련
"6354", "5704", "5720", # 건설기획팀
# 이하 그 외
"5707", "6084", "6161", "5934", "5218", "6110", "4632", "5104", "5892", "6335", "6353", "6284", "6281", "6427", "6012", "6360", "6122", "5994", "6385", "6310", "6389", "5516", "6455", "5903", "5919",
"6457", "6036", "5308", "6469", # 마케팅팀 홍일석, 이호창, 최혁, 김민경
"6034", "6333", # 공정상생팀 유동석, 김관빈
"5634", # 맹은주
"3991", "4154", "4826", "5381", "5477", "5555", "5634", "5673", "5720", "5892", "5969", "6033", "6253", "6376", "6414", "6453", "6456", "6460", "6045", "6448", # 경영현안 회의
"3684", "3805", "3872", "3911", "3991", "4009", "4057", "4078", "4540", "4598", "6446"]))] # PD

print("Created users;")
with server.auth.sign_in(tableau_auth):
    for _, row in users_to_create.iterrows():
        # `site_role` 결정
        if row["num"] not in [i.name for i in all_users]:
            print(row["num"], row["name"], row["dept"], row["job"])
            if row["num"] in ["4100435"]: # 이재호
                user = TSC.UserItem(row["num"], "SiteAdministratorCreator")
            elif row["num"] in ["4001380"]: # 김아현
                user = TSC.UserItem(row["num"], "Creator")
            else:
                user = TSC.UserItem(row["num"], "Viewer")                
            user = server.users.add(user)
        else:
            user = [i for i in all_users if i.name == row["num"]][0]
            
        if row["job"] in ["회장", "부회장", "대표이사", "본부장", "팀장", "실장", "그룹장", "부문장", "단장", "PL", "PM", "TFT장", "건축소장", "토목소장"]:
            user.fullname = f"{row['name']}({row['job']})"
        else:
            user.fullname = f"{row['name']}({row['dept']})"
        user.email = row["email"]
        user = server.users.update(user)
    
print("Removed users;")
with server.auth.sign_in(tableau_auth):
    for user in all_users:
        if user.name not in users_to_create["num"].tolist() + ["6415", "4686", "Test", "goodcen"]: # 정경구, 도기탁, 랩스, 굿센
            temp = hr[hr["num"]==user.name]
            if len(temp) > 0:
                print(temp.iloc[0]["num"], temp.iloc[0]["name"], temp.iloc[0]["dept"], temp.iloc[0]["job"])
                server.users.remove(user.id)

print("All completed!")

Created users;
4982 김상현 지개-남산간연결도로민간투자사업 토목소장
Removed users;
4064 박상우 용현학익1블럭부지조성공사 토목담당
All completed!


# Add Users to Groups

In [4]:
# all_users.csv
with server.auth.sign_in(tableau_auth):
    all_users, _ = server.users.get(req_options=req_opts)
all_users_df = [(user.id, user.name, user.fullname, user.site_role) + empls[user.name] if user.name in empls.keys() else (user.id, user.name, user.fullname, user.site_role, None, None, None, None) for user in all_users]
all_users_df = pd.DataFrame(all_users_df, columns=["user_id", "user_name", "user_fnm", "site_role", "hq", "dept", "job", "name"])
all_users_df = all_users_df.sort_values(["user_fnm"])
all_users_df.to_csv("./all_users_df.csv", encoding="euc-kr")

# all_groups.csv
with server.auth.sign_in(tableau_auth):
    all_groups, _ = server.groups.get(req_options=req_opts)
all_groups = [(group.name, group.id) for group in all_groups]
all_groups = pd.DataFrame(all_groups, columns=["group", "group_id"])

# 관리자
users_to_alloc = hr[(hr["num"].isin(["6363", "5899", "5947"])) | ((hr["job"]=="팀장") & (hr["dept"]=="디지털플랫폼팀"))]
tar_grp = "6e2fc4f0-46f2-4114-8b25-341b38af9f7d"
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    tar_grp = [i for i in groups if i.id == tar_grp][0]
    for _, row in users_to_alloc.iterrows():
        user_id = all_users_df.loc[all_users_df["user_name"]==row["num"], "user_id"].tolist()[0]
        try:
            server.groups.add_user(tar_grp, user_id)
        except:
            continue

# 회장/부회장/대표이사/본부장
users_to_alloc = hr[(hr["job"]=="회장") | (hr["job"]=="부회장") | (hr["job"]=="대표이사") | (hr["job"]=="본부장") | ((hr["job"]=="팀장") & (hr["dept"]=="디지털플랫폼팀")) | (hr["dept"]=="상근고문") | (hr["num"].isin(["6032", "6294", "6119", "6111", "6403", "1101291", "6048", "5634"]))] # ERP 관련
tar_grp = "f827e0fe-ff32-4443-9c40-701c94a13ba1"
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    tar_grp = [i for i in groups if i.id == tar_grp][0]
    for _, row in users_to_alloc.iterrows():
        user_id = all_users_df.loc[all_users_df["user_name"]==row["num"], "user_id"].tolist()[0]
        try:
            server.groups.add_user(tar_grp, user_id)
        except:
            continue

# 팀장/실장/그룹장/부문장/단장/PL/PM/TFT장
users_to_alloc = hr[((hr["sub"] != "정보기술/CS외주") & (hr["job"]=="팀장")) | (hr["job"]=="실장") | (hr["job"]=="그룹장") | (hr["job"]=="부문장") | (hr["job"]=="단장") | (hr["job"]=="PL") | (hr["job"]=="PM") | (hr["job"]=="TFT장")]
tar_grp = "39dc2d08-d291-4534-97d5-b1a6b3b6f67c"
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    tar_grp = [i for i in groups if i.id == tar_grp][0]
    for _, row in users_to_alloc.iterrows():
        user_id = all_users_df.loc[all_users_df["user_name"]==row["num"], "user_id"].tolist()[0]
        try:
            server.groups.add_user(tar_grp, user_id)
        except:
            continue
        
# 건축소장/토목소장
users_to_alloc = hr[(hr["job"]=="건축소장") | (hr["job"]=="토목소장")]
tar_grp = "05fa4bd0-e1df-4971-bb89-4e39f24782d7"
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    tar_grp = [i for i in groups if i.id == tar_grp][0]
    for _, row in users_to_alloc.iterrows():
        user_id = all_users_df.loc[all_users_df["user_name"]==row["num"], "user_id"].tolist()[0]
        try:
            server.groups.add_user(tar_grp, user_id)
            print(f"건축소장/토목소장: {row['num']} {row['name']}")
        except:
            continue

# HR혁신팀
users_to_alloc = hr[(hr["num"].isin(["4100222"])) | ((hr["dept"]=="HR혁신팀") & (hr["sub"]=="일반직"))]
tar_grp = "1ac1f9c5-d4b9-47d9-8c40-618df8f87f6b"
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    tar_grp = [i for i in groups if i.id == tar_grp][0]
    for _, row in users_to_alloc.iterrows():
        user_id = all_users_df.loc[all_users_df["user_name"]==row["num"], "user_id"].tolist()[0]
        try:
            server.groups.add_user(tar_grp, user_id)
            print(f"HR혁신팀: {row['num']} {row['name']}")
        except:
            continue

# 경영현안 회의
users_to_alloc = hr[((hr["job"]=="부회장") | (hr["job"]=="대표이사") | (hr["job"]=="본부장")) | (hr["dept"]=="상근고문") | (hr["num"].isin(["3991", "4154", "4826", "5381", "5477", "5555", "5634", "5673", "5720", "5892", "5969", "6033", "6253", "6376", "6414", "6453", "6456", "6460", "3684", "3805", "3872", "3911", "3991", "4009", "4057", "4078", "4540", "4598", "6446", "6415", "4686", "5704", "6045", "6012", "4001380", "6036", "6442"]))]
tar_grp = "388136df-5404-4b2d-8805-0276a6c3080a"
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    tar_grp = [i for i in groups if i.id == tar_grp][0]
    for _, row in users_to_alloc.iterrows():
        user_id = all_users_df.loc[all_users_df["user_name"]==row["num"], "user_id"].tolist()[0]
        try:
            server.groups.add_user(tar_grp, user_id)
            print(f"경영현안 회의: {row['num']} {row['name']}")
        except:
            continue
            
# 본부장 회의 일반 사용자
users_to_alloc = hr[(hr["num"].isin(["6427", "6457", "6036", "6335", "4001380", "4100435", "6281", "4001341", "4001360", "4001386", "5653", "6012", "6360", "6122", "5994", "6385", "6389", "5516", "6455", "5835", "5903", "5919", "6354", "5704", "5720", "4632", "5308",
"5888", # 경영분석팀 김주완
"6034", "6333"]))] # 공정상생팀 유동석, 김관빈
tar_grp = "0ef096dd-2c99-4473-b80b-855d1c0b29e1"
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    tar_grp = [i for i in groups if i.id == tar_grp][0]
    for _, row in users_to_alloc.iterrows():
        user_id = all_users_df.loc[all_users_df["user_name"]==row["num"], "user_id"].tolist()[0]
        try:
            server.groups.add_user(tar_grp, user_id)
            print(f"본부장 회의 일반 사용자: {row['num']} {row['name']}")
        except:
            continue
            
# 경영 현황 일반 사용자
users_to_alloc = hr[((hr["dept"]=="경영분석팀") & (hr["sub"]=="일반직") & (hr["job"]=="팀원") & (~hr["num"].isin(["4113"]))) | (hr["num"].isin(["6360", "6122", "5994", "6385", "6310", "6389", "5516", "6455", "5835", "5903", "5919", "6354", "5704", "5720", "6469", "5308",
"6034", "6333"]))] # 공정상생팀 유동석, 김관빈
tar_grp = "91d8104c-e669-4ea2-a5fa-27b6be938858"
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    tar_grp = [i for i in groups if i.id == tar_grp][0]
    for _, row in users_to_alloc.iterrows():
        user_id = all_users_df.loc[all_users_df["user_name"]==row["num"], "user_id"].tolist()[0]
        try:
            server.groups.add_user(tar_grp, user_id)
            print(f"경영 현황 일반 사용자: {row['num']} {row['name']}")
        except:
            continue

# HR 현황 일반 사용자
users_to_alloc = hr[(hr["num"].isin(["6385", "6385", "6122", "6455", "5903", "5919", "6354", "5704", "5720", "6284", "5104", "6469", "5308",
"6034", "6333"]))] # 공정상생팀 유동석, 김관빈
tar_grp = "fef23089-cfd7-403d-8982-894afac13b32"
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    tar_grp = [i for i in groups if i.id == tar_grp][0]
    for _, row in users_to_alloc.iterrows():
        user_id = all_users_df.loc[all_users_df["user_name"]==row["num"], "user_id"].tolist()[0]
        try:
            server.groups.add_user(tar_grp, user_id)
            print(f"HR 현황 일반 사용자: {row['num']} {row['name']}")
        except:
            continue
            
# 영업 현황 일반 사용자
users_to_alloc = hr[((hr["dept"]=="경영분석팀") & (hr["sub"]=="일반직") & (hr["job"]=="팀원") & (~hr["num"].isin(["4113"]))) | (hr["num"].isin(["5218", "6337", "4632", "6110", "6122", "5994", "6385", "6310", "6389", "5516", "6455", "5835", "5903", "5919", "6354", "5704", "5720", "6284", "5934", "5104", "6469",
"6457", "6036", "5308", # 마케팅팀 홍일석, 이호창, 최혁
"6034", "6333"]))] # 공정상생팀 유동석, 김관빈
tar_grp = "0715f81d-9385-4fc8-8c2a-76f8b9481dac"
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    tar_grp = [i for i in groups if i.id == tar_grp][0]
    for _, row in users_to_alloc.iterrows():
        user_id = all_users_df.loc[all_users_df["user_name"]==row["num"], "user_id"].tolist()[0]
        try:
            server.groups.add_user(tar_grp, user_id)
            print(f"영업 현황 일반 사용자: {row['num']} {row['name']}")
        except:
            continue

# 현장 현황 일반 사용자
users_to_alloc = hr[((hr["dept"]=="경영분석팀") & (hr["sub"]=="일반직") & (hr["job"]=="팀원") & (~hr["num"].isin(["4113"]))) | (hr["num"].isin(["5947", "6353", "6360", "6122", "5994", "6385", "6310", "6389", "5516", "6455", "5835", "5903", "5919", "6354", "5704", "5720", "6284", "5934", "5104", "6469", "5308",
"6034", "6333"]))] # 공정상생팀 유동석, 김관빈
tar_grp = "fcace1dd-cfa8-4f64-8b20-803c739e1e3c"
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    tar_grp = [i for i in groups if i.id == tar_grp][0]
    for _, row in users_to_alloc.iterrows():
        user_id = all_users_df.loc[all_users_df["user_name"]==row["num"], "user_id"].tolist()[0]
        try:
            server.groups.add_user(tar_grp, user_id)
            print(f"현장 현황 일반 사용자: {row['num']} {row['name']}")
        except:
            continue
            
# 직무정보 일반 사용자
users_to_alloc = hr[(hr["num"].isin(["5892"]))] # 서유민
tar_grp = "b2e29db6-c54f-45ce-b021-31596d75efc1"
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    tar_grp = [i for i in groups if i.id == tar_grp][0]
    for _, row in users_to_alloc.iterrows():
        user_id = all_users_df.loc[all_users_df["user_name"]==row["num"], "user_id"].tolist()[0]
        try:
            server.groups.add_user(tar_grp, user_id)
        except:
            continue
            
print("All completed!")

건축소장/토목소장: 4982 김상현
All completed!


# Modify User Information

In [8]:
with server.auth.sign_in(tableau_auth):
    all_users, _ = server.users.get(req_options=req_opts)
    for user in all_users:
        if user.name in ["5899"]:
            user = server.users.update(user, password="secretpassword")
print("All Completed!")

All Completed!


# Remove Users

In [ ]:
# 발령 이력 활용

# conn = pymssql.connect(server="133.186.215.165", database="HDCMART", user="hdcmart_user", password=password, charset="UTF8")
# query = """
# SELECT id_sabun, ds_hname, ds_order1, ds_order2, ds_deptt, ds_position, ds_emptype, dt_order, dt_orderend 
# FROM DATAMART_DAAV_ORDER
# WHERE cd_corp = 'A101' AND dt_order > '20190101'
# """
# hist = pd.read_sql(query, conn)
# hist.columns = hist.columns.str.lower()
# for col in hist.columns:
#     if pd.api.types.is_string_dtype(hist[col]):
#         hist[col] = hist[col].str.encode("latin-1").str.decode("euc-kr")

In [ ]:
# conn = pymssql.connect(server="133.186.215.165", database="HDCMART", user="hdcmart_user", password=pw1, charset="UTF8")
# query = """
# SELECT id_sabun, ds_hname, ds_bonbu, ds_deptt, ds_emptype, ds_position, ds_occupation, ds_retire
# FROM DATAMART_DAAV_BASEINFO_DETAIL
# """
# hr2 = pd.read_sql(query, conn)

# hr2.columns = hr2.columns.str.lower()
# for col in hr2.columns:
#     if pd.api.types.is_string_dtype(hr2[col]):
#         hr2[col] = hr2[col].str.encode("latin-1", errors="ignore").str.decode("euc-kr", errors="ignore")

# hr2[(hr2["ds_retire"]=="재직") & ((hr2["ds_position"]=="회장") | (hr2["ds_position"]=="부회장") | (hr2["ds_occupation"]=="대표이사") | (hr2["ds_occupation"]=="본부장") | (hr2["ds_occupation"]=="팀장") | (hr2["ds_occupation"]=="실장") | (hr2["ds_occupation"]=="그룹장") | (hr2["ds_occupation"]=="부문장") | (hr2["ds_occupation"]=="단장") | (hr2["ds_occupation"]=="PL") | (hr2["ds_occupation"]=="PM") | (hr2["ds_occupation"]=="TFT장") | (hr2["ds_occupation"]=="건축소장") | ((hr2["ds_deptt"]=="HR혁신팀") & (hr2["ds_emptype"]=="일반직")))]

Create Fundamental Datasets

In [ ]:
# # user_group.csv
# user_group = list()
# with server.auth.sign_in(tableau_auth):
#     users, pagination_item = server.users.get(req_options=req_opts)
#     for user in users:
#         pagination_item = server.users.populate_groups(user, req_options=req_opts)
#         user_group.extend([(user.id, user.name, user.fullname, user.site_role) + empls[user.name] + (group.name,) if user.name in empls.keys() else (user.id, user.name, user.fullname, user.site_role, None, None, None, None, group.name) for group in user.groups if group.name != "All Users"])
# user_group = pd.DataFrame(user_group, columns=["user_id", "user_name", "user_fnm", "site_role", "hq", "dept", "job", "name", "group"])

# user_group.to_csv("D:/Github/Work/Tableau/user_group.csv", index=False, encoding="euc-kr")

# # group_user.csv
# group_user = list()
# with server.auth.sign_in(tableau_auth):
#     groups, pagination_item = server.groups.get(req_options=req_opts)
#     for group in groups[1:]:
#         pagination_item = server.groups.populate_users(group, req_options=req_opts)
#         group_user.extend([(group.name,) + empls[user.name] + (user.site_role,) if user.name in empls.keys() else (group.name, None, None, None, user.name, user.site_role) for user in group.users])

# group_user = pd.DataFrame(group_user, columns=["group", "hq", "dept", "job", "name", "site_role"])

# group_user.to_csv("D:/Github/Work/Tableau/group_user.csv", index=False, encoding="euc-kr")

# # user_workbook
# user_wb = list()
# with server.auth.sign_in(tableau_auth):
#     users, pagination_item = server.users.get(req_options=req_opts)
#     for user in users:
#         pagination_item = server.users.populate_workbooks(user, req_options=req_opts)
#         user_wb.extend([(user.id, user.name, user.fullname, user.site_role) + empls[user.name] + (wb.name,) if user.name in empls.keys() else (user.id, user.name, user.fullname, user.site_role, None, None, None, None, wb.name) for wb in user.workbooks if wb.project_name=="HDC현대산업개발"])
# user_wb = pd.DataFrame(user_wb, columns=["user_id", "user_name", "user_fnm", "site_role", "hq", "dept", "job", "name", "wb"])

# user_wb.to_csv("D:/Github/Work/Tableau/user_wb.csv", index=False, encoding="euc-kr")

# # have_account
# have_account = hr[["num", "hq", "dept", "name", "sub", "job", "rank", "email"]]
# have_account["have"] = have_account["num"].apply(lambda x:True if str(x) in all_users["user_name"].tolist() else False)

# have_account.to_csv("D:/Github/Work/Tableau/have_account.csv", index=False, encoding="euc-kr")

# print("All completed!")

In [ ]:
# hr2 = pd.read_excel("D:/Github/Work/Tableau/직원 명단_조직 기준_211130.xlsx")
# hr2["성명"] = hr2["성명"].apply(lambda x:x.replace(" ", ""))
# hr2 = hr2[["사번", "EMAIL", "본부명(조직단위기준)", "조직명", "하위그룹", "직무", "직급", "성명"]]
# hr2 = hr2.rename({"사번":"num", "EMAIL":"email", "본부명(조직단위기준)":"hq", "조직명":"dept", "하위그룹":"sub", "직무":"job", "직급":"rank", "성명":"name"}, axis=1)
# hr2 = hr2.sort_values(by=["job", "rank", "hq", "dept"], axis=0)
# hr2["num"] = hr2["num"].astype("str")

# hr2.loc[hr2["num"]=="1", "num"] = "0001"